## Test Create CHCs

In this file we test the creation of CHCs from a given program.

First we import the necessary libraries and set up the Z3 solver.

In [1]:
from z3 import And, Ints, Array, IntSort
import z3
import graphviz
z3.set_param(proof=True)
from sw.verify.stasm_to_chcs import create_checs

Now let's create a simple program and generate the CHCs for it.
This program is taken straight out of the class presentation.
It is a sanity test case for our implementation.
All it does is `stack[sp-2] + stack[sp-1] * 13`.  

In [9]:
program = [("PUSH", 13),
              ("POP", 2),
              ("ALU", "MUL"),
              ("POP", 2),
              ("ALU", "ADD"),
              ("POP", 1)]

In our case we are going to set 
`stack[0] = 1, stack[1] = 5`
and test that the output is `1 + 5 * 13`

In [13]:
stack = Array("stack", IntSort(), IntSort())
sp, r0, r1 = Ints('sp r0 r1')
chcs = create_checs(pre_condition=And(stack[0] == 1, stack[1] == 5, sp == 2),
             input_vars=[],
             program=program,
             post_condition=stack[sp] == 1 + 5 * 13)
print(chcs.solve())

[U6 = [else ->
       Or(Exists(k!4,
                 And(k!4[1] == 5,
                     k!4[0] == 1,
                     Var(4) == Var(2)[0],
                     Var(5) == k!4[0],
                     Var(2) ==
                     Store(Store(Store(k!4, 2, 13),
                                 1,
                                 13*k!4[1]),
                           0,
                           13*k!4[1] + Var(5)),
                     Var(3) == 0)),
          Exists([k!2, k!1],
                 And(Exists(k!4,
                            And(k!4[1] == 5,
                                k!4[0] == 1,
                                Var(7) == k!4[0],
                                k!1 == 13*k!4[1],
                                Var(4) ==
                                Store(Store(Store(k!4,
                                        2,
                                        13),
                                        1,
                                        13*k!4[1]),
    

Now lets test the find max program from milestone 0
First, import some macros:

In [4]:
def READ_FROM_ARRAY():
    """
    Pops the top as index and then base addr and push the value at that index from the array at addr
    :return:
    """
    return [
        ("POP", 2),  # r0 = index, r1 = base addr
        ("ALU", "ADD"),  # push base addr + index
        ("POP", 1),   # r0 = base addr + index
        ("LOAD", 0),  # push value at base addr + index
    ]

Now writing the program:

In [5]:
program_find_max = [
    ("PUSH", 0),
    ("DUP", 2),
    *READ_FROM_ARRAY(),
    # Stack is now: [arr_addr, length, a[0]]
    # Now let's define max = a[0]
    ("DUP", 0),
    # Stack is now: [arr_addr, length, a[0], mx=a[0]]
    # Now after we saved a[0] on stack we can use this mem for i
    # lets set i = 1 to memory address &a[0]
    ("PUSH", 1),
    ("DUP", 4),
    ("POP", 2),
    ("STOR", 0),
    "CHECK_COND:",
    # First put i in r0 and n in r1
    ("DUP", 3),
    ("POP", 1),
    ("LOAD", 0),
    ("DUP", 3),
    ("POP", 2),
    # Check i < n
    ("ALU", "LT"),
    ("POP", 1),
    ("JNZ", "LOOP_BODY"),
    ("JMP", "END"),
    "LOOP_BODY:",
    # read i from mem and put on stack
    ("DUP", 3),
    ("POP", 1),
    ("LOAD", 0),
    # put base addr on stack
    ("DUP", 4),
    # Put a[i] on stack
    *READ_FROM_ARRAY(),
    # Put mx on stack
    ("DUP", 1),
    # now stack is [base_addr, n, a[0], mx, a[i], mx]
    ("POP", 2),
    ("ALU", "LT"),
    # now stack is [base_addr, n, a[0], mx, (result a[i] < mx)]
    ("POP", 1),
    ("JNZ", "INC_I"),
    ("JMP", "UPDATE"),
    "INC_I:",
    # Put i on stack
    ("PUSH", 0),
    ("DUP", 4),
    *READ_FROM_ARRAY(),
    ("PUSH", 1),
    ("POP", 2),
    ("ALU", "ADD"),
    ("DUP", 4),
    ("POP", 2),
    ("STOR", 0),
    ("JMP", "CHECK_COND"),
    "UPDATE:",
    ("POP", 1),  # remove old mx from stack
    # Push base addr
    ("DUP", 2),
    ("POP", 1),
    ("LOAD", 0),
    # Now i is on top of stack
    ("DUP", 3),
    *READ_FROM_ARRAY(),  # so now a[i] is where mx was on stack so mx = a[i]
    ("JMP", "INC_I"),
    "END:",
    # Restore a[0]:
    # Put a[0] on stack (we saved it before):
    ("DUP", 1),
    # Put base_addr on stack:
    ("DUP", 4),
    ("POP", 2),
    ("STOR", 0),
    ("POP", 2),
    ("POP", 1),
    ("POP", 1)
]

Now creating the verification 

In [16]:
stack = Array("stack", IntSort(), IntSort())
memory = Array("memory", IntSort(), IntSort())
sp, r0, r1 = Ints('sp r0 r1')
chcs = create_checs(pre_condition=And(stack[0] == 0, stack[1] == 4, sp == 2,
                                      memory[0] == 2, memory[1] == 8, memory[2] == 10, memory[3] == 3),
             input_vars=[],
             program=program_find_max,
             post_condition=And(sp == 0,
                                memory[0] == 2, memory[1] == 8, memory[2] == 10, memory[3] == 3,
                                r1 == 10))
print(chcs.solve())

[U31 = [else ->
        Or(Exists([k!7, k!6, k!4],
                  And(Not(Var(3)[k!4] + -1*k!7[-2 + k!6] <=
                          1),
                      Not(k!7[-2 + k!6] >= 3),
                      Not(k!7[-3 + k!6] + -1*k!7[-2 + k!6] <=
                          1),
                      Var(5) == 1 + k!6,
                      Var(6) ==
                      Var(3)[Store(Store(Store(Store(k!7,
                                        k!6,
                                        Var(3)[k!4]),
                                        1 + k!6,
                                        Store(k!7,
                                        k!6,
                                        Var(3)[k!4])[-2 +
                                        k!6]),
                                    k!6,
                                    Var(3)[Store(Store(k!7,
                                        k!6,
                                        Var(3)[k!4]),
                                        